<a href="https://colab.research.google.com/github/mtyt/classevy/blob/main/Recipes_for_the_planet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Our World In Data: Environmental Impacts of Food Production
The brilliant [Our World In Data](https://ourworldindata.org) published an article that shows how what we eat impacts the environment. For a number of foods - I think about 200 - they collected the CO2 emissions, land use, water use, [eutrophication](https://en.wikipedia.org/wiki/Eutrophication) per kg but also per 100 g of protein, 100 g of fat and per 1000 kcal. This way, they created some interesting grahps which yield insights into which foods are better for the planet than others, and which are very bad ([BEEF!](https://ourworldindata.org/explorers/food-footprints?facet=none&hideControls=false&Commodity+or+Specific+Food+Product=Commodity&Environmental+Impact=Carbon+footprint&Kilogram+%2F+Protein+%2F+Calories=Per+kilogram&By+stage+of+supply+chain=false&country=Bananas~Beef+%28beef+herd%29~Beef+%28dairy+herd%29~Cheese~Eggs~Lamb+%26+Mutton~Milk~Maize~Nuts~Pig+Meat~Peas~Potatoes~Poultry+Meat~Rice~Tomatoes~Wheat+%26+Rye~Tofu+%28soybeans%29~Prawns+%28farmed%29)).

However, this isn't a very practical way of deciding what you will eat for dinner tonight! You can think: "oh perhaps I will have some beer and some peas!" and then look up the environmental impact. But how many kilograms of beer and peas will you need for a healthy diet?
That's where this Notebook comes in.
I wanted to reverse the question and solve the problem: I want to consume just enough food to satisfy my nutritional needs (kcal, fat and protein) at the lowest environmental impact possible, what should I eat?

In order to solve this question, I'm using a [genetic algorithm (GA)](https://en.wikipedia.org/wiki/Genetic_algorithm), which I implemented in a Python package, available on GitHub, called [`optime`](https://github.com/mtyt/optime), to optimize your diet towards minimal environmetal impact, while satisfying your healthy nutritional needs!

- [First](#one-food), I will take a simplified case where you're going to eat only 1 type of food every day. Which food should it be to consume enough calories, fat and protein, while having the minimum environmental impact possible?

- [Second](#all-food), we will use the power of all of the foods and combine them to find the ultimate recipe that satisfies our hunger while minimizing environmental impact!


## How to use:
If you are not familiar with Python, Jupyter Notebooks and/or Google Colab: no worries! You don't need to understand the code to execute it. Just hit the play-button on the left of each cell or hit shift-enter or click Runtime > Run all. If you're still having problems, don't hesitate to contact me for help.

The first time running this Notebook after opening, it might take a while to install some dependencies. Please be patient!

## Sources:
- [Our World In Data: Environmental Impacts of Food Production](https://ourworldindata.org/environmental-impacts-of-food):
Hannah Ritchie and Max Roser (2022) - "Environmental Impacts of Food Production". Published online at OurWorldInData.org.
- Optime: a simple optimizer package, available on [GitHub](https://github.com/mtyt/optime)


## Disclaimer
Unfortunately, the data available on the website of OWID does not mention any sugar contents in any of the foods, so it's perfectly possible that the diets I'm proposing are actually life-threatening. So please don't use this Notebook as actual dietary advice!
Also, I didn't look at cost.
But, provide me the data and I'll add it to the equation!

## About
Do what you want with this Notebook but it would be nice if you mentioned it was made by Maarten Tytgat (maarten.tytgat@gmail.com), who has a website: www.futiledevices.be

In [ ]:
# At every new Runtime, we unfortunately have to install some things. Don't worry,
# this doesn't put anything on your computer, it's all in the cloud.
!pip install --upgrade pip
!pip install 'optime @ git+https://github.com/mtyt/optime@8439669e9028419082028c2c363bb264892dc9ff'

In [ ]:
# Some standard python stuff:
from functools import partial
import pandas as pd
import re
import numpy as np
from functools import cached_property
import matplotlib.pyplot as plt
from optime.ga import Population, child

In [ ]:
# Loading the data and printing the first few lines.
df = pd.read_csv("https://drive.google.com/uc?export=download&id=1B68Sg_OQblBxJkmqP3lLXSJ2C23tFEJy")
df = df.set_index('Entity')
df[0:3]

# What should I eat to keep myself and the planet alive?
According to some sources, a person needs 2000 kcal, 72g fat and 50g protein a day. If you think this is different for you, please go ahead and change the numbers before executing the cell (and the next cells).
Again, note how the program does not care about your sugar intake!

In [ ]:
target_kcal = 2000
target_fat = 72
target_protein = 50

The original data only mentions the environmental impact (emissions, land use etc) per kg or kcal etc of food, but not the kcal/kg or fat/kg etc. Those can be obtained by dividing one number by the other. But I noticed that the results differ slightly depending on which criterium (Emissions, Land use, Eutrophication or Water withdrawals) I use.


Different impact criteria yield different amount of nutrients per kg. Let's for example look at protein content for Ale:

In [ ]:
food = 'Ale'
prot_1 = 100/df.at[food,'Emissions per 100 grams of protein']*df.at[food,'Emissions per kilogram']
prot_2 = 100/df.at[food,'Eutrophication per 100 grams of protein']*df.at[food, 'Eutrophication per kilogram']
print(f'grams of protein per kg of {food} based on Emissions = {prot_1}')
print(f'grams of protein per kg of {food} based on Eutrophication = {prot_2}')

So, I will take the average of the nutrient per kg as given by each of the criteria and use that from now on:

In [ ]:
criteria = ['Emissions', 'Land use', 'Eutrophication', 'Water withdrawals']
nutrients = ['1000 kilocalories', '100 grams of protein', '100 grams of fat']

new_cols = []
for nut in nutrients:
  for crit in criteria:
    new_col = nut + '/kg based on ' + crit
    new_cols.append(new_col)
    df[new_col] = df[crit + ' per kilogram'] / df[crit + ' per ' + nut]
  df[nut + '/kg'] = df[[(nut + '/kg based on ' + crit) for crit in criteria]].mean(axis='columns')
print("Showing the new columns only, with the nutrients per kg based on the different criteria:")
df[new_cols]

In [ ]:
print("Showing the new columns of the average nutrients per kg:")
df[[nut + '/kg' for nut in nutrients]]

In [ ]:
# Now we don't need those new columns anymore and we'll just the average from now on.
# Let's also remove some columns we won't use.
remove_cols = new_cols + [
    'Year',
       'biodiversity_kg', 'biodiversity_1000kcal', 'biodiversity_100gprotein',
       'biodiversity_100gfat'
]
df = df.drop(columns=remove_cols)
df

# Preparation: define a recipe as a vector of kilograms per food
Take a random vector of food masses per food and calculate the total nutritional value and environment impact. We'll re-use this function later on.
The table shows for every food how many kilograms we have, and what is the nutritional value and environmental impact of that amount of food.

In [ ]:
kg_vec = np.random.uniform(0,1,len(df)) # every food can have 0-1kg - no one said this was going to be a realistic example

def df_recipe(kg_vec):
    # copy the df and keep only the food names and nutrients and criteria, add a column for the kg vector,
    # then calculate the total nutrients and env impact based on the kg value:
    df_case = df[[nut + '/kg' for nut in nutrients]+[crit + ' per kilogram' for crit in criteria]].copy()
    df_case['kg'] = kg_vec.tolist()
    df_case['kcal'] = df_case['1000 kilocalories/kg']*df_case['kg']*1000
    df_case['g protein'] = df_case['100 grams of protein/kg']*df_case['kg']*100
    df_case['g fat'] = df_case['100 grams of fat/kg']*df_case['kg']*100

    for crit in criteria:
        df_case[crit] = df_case[crit + ' per kilogram']*df_case['kg']

    # clean up the df, to contain only the data we want to see:
    df_clean = df_case[['kg', 'kcal', 'g protein', 'g fat'] + criteria].copy()

    return df_clean

df_clean = df_recipe(kg_vec)
df_clean

By summing the values in the columns, we can quickly see the total nutritional value and environmental impact if we were to eat the whole table.

In [ ]:
df_clean.sum()

<a name="one-food"></a>
# Case 1: You can only eat 1 type of food a day
For any food, how many kg do you need in order to have enough, and what is the environmental impact?
The only condition is that each nutrient is consumed at least as much as the recommended daily dose:
- target_kcal = 2000
- target_fat = 72
- target_protein = 50

(Or whatever you changed to numbers to)

So we determine the amount of kg for each food based on this procedure:
Calculate the kg you'd need to meet the target for each nutrient separately, then take the maximum. Note that the resulting table is not a list of everything we should eat in 1 day, but a list of how much we should eat of each food if we only ate that food that day!

In [ ]:
df_case_1_food = df.copy()
# calculate an amount of kg you would need of the food to guarantee the recommended intake of kcal, protein and fat:
df_case_1_food['mult_kcal'] = target_kcal/df_case_1_food['1000 kilocalories/kg']/1000
df_case_1_food['mult_protein'] = target_protein/df_case_1_food['100 grams of protein/kg']/100
df_case_1_food['mult_fat'] = target_fat/df_case_1_food['100 grams of fat/kg']/100
# Take the max of the above 3 multipliers to ensure enough intake!
kg_vec = df_case_1_food[['mult_kcal', 'mult_protein', 'mult_fat']].max(axis='columns')
df_case_1_food['kg'] = kg_vec
df_clean = df_recipe(kg_vec)
print("The resulting DataFrame shows how much of each food (only that food) to eat daily, and the\
 resulting nutrients and environmental impact:")
df_clean

Note that thanks to its high fat contents, you'd only need to drink about 3 liters of beer to get your minimum requirements, versus more than 70 liters of wine!

Since we restrict ourselves to only 1 food, there's not much flexibility in terms of balancing your calories, fat and protein intake. Neither is there any way to adjust the relative contributions to Emissions, Land use, Eutrophication and Water withdrawels.

However, we can sort the results by one of the criteria in order to see which food we should eat (and only that food) to minimize our impact there:

In [ ]:
crit = 'Emissions'
print(f"The DataFrame sorted by {crit}:")
df_clean.sort_values(crit)

Turns out it's Almond butter for lowest Emissions! And you only need about 340 grams of it. Cheap! Unsuprisingly, Almonds come in close second. Actually you might wonder what the difference is. Anyhow. Let's also print the best foods in terms of other impact factors:

In [ ]:
for crit in criteria:
  row = df_clean.sort_values(crit).iloc[0]
  print(f"You only have to eat {row['kg']:.2} kg of {row.name} for miminum {crit}.")

So that's interesting: Almonds, pumkin seeds and beans are pretty good for the environment. But, we have different optima for different impact criteria. Moreover, by eating only almond butter, you will eat a bit too much protein and too much fat! Surely we can do better by applying the art of throwing together a bunch of different ingredients!

<a name="all-food"></a>
# Case 2: Optimize the food vector
Can we find the ultimate food combination that satisfies our hunger and minizes environmental impact?
We need to either weigh the 4 impact criteria, or consider them independently and look at the Pareto-optimum.
When we compare a number of food-vectors:
what are the non-dominated items? That means they are at least the best on 1 criterium and not worse on the others.

Note that we could also optimize for low impact and high nutritional
  value but those inherently conflict with each other. Increasing the total kg
  value of a food vector will always increase its nutritional value but also
  its environmental impact. Here, we're only interested in getting *just enough*
  nutrition.

### Let's first make a class to define the food vector and its scores. Let's call it... Recipe!

In [ ]:
def col_to_var(col):
    """Function to turn a DataFrame column name into a variable name, replacing
    capitals by lowercase and spaces by underscores.
    """
    return re.sub('\W|^(?=\d)','_', col).lower()
criteria_vars = [col_to_var(col) for col in criteria]

# Little helper function to set smallest elements to 0:
def keep_k_largest(vec, k):
  """In the vector vec, keep the k largest numners, set others to 0.
  But vector keeps same length.
  """
  if not isinstance(vec, np.ndarray):
    vec = np.array(vec)
  small_ind = np.argpartition(vec, -k)[:-k]
  vec[small_ind] = 0
  return vec

class Recipe():
    """ A Recipe will be the individual in the genetic algorithm. The best Recipe
    is the one that has the lowest environmental impact and just enough nutritional
    content.

    A Recipe can be initialized without any arguments, in which case it will be
    created with a random vector for kg values.
    """
    def __init__(self, kg_vec=None, max_ingredients=None):
        if kg_vec is None:
            if max_ingredients:
                vec_len = max_ingredients
            else:
                vec_len = len(df)
            kg_vec = np.zeros(len(df))
            # choose which indices will be assigned a value:
            choisi = np.random.choice(np.arange(len(kg_vec)), vec_len, replace=False)
            # I've manually finetuned the upper limit for kg_vec in order to get 
            # close to target nutrients for a random vector
            kg_vec[choisi] = np.random.uniform(0,0.008*len(df)/vec_len, vec_len)
        self.max_ingredients = max_ingredients
        self.kg_vec = kg_vec # use the setter

    @property
    def kg_vec(self):
        return self._kg_vec
    
    @kg_vec.setter
    def kg_vec(self, x):
        if self.max_ingredients:
            # set the smallest values to zero, keeping only max_ingredients
            x = keep_k_largest(x, self.max_ingredients)
        self._kg_vec = x
        # when the setter is called, these cached_property must be updated:
        list_of_dependent_properties = ['df',
                                        'summary',
                                        'impact',
                                        'nutrients',
                                        'enough',
                                        'not_enough'
                                        ]
        for prop in list_of_dependent_properties:
            if prop in  self.__dict__:
                delattr(self,prop)

    @cached_property
    def df(self):
      # re-use function from above and sort by kg:
      return df_recipe(self.kg_vec).sort_values('kg', ascending=False)
    
    @cached_property
    def summary(self):
        return self.df.sum()
    
    @cached_property
    def impact(self):
        return self.summary[criteria]
    
    def get_crit(self, crit):
        return self.impact[crit]
    
    for crit, crit_var in zip(criteria, criteria_vars):
        fget_crit = partial(get_crit, crit=crit)
        vars()[crit_var] = property(fget=fget_crit)
    
    @cached_property
    def nutrients(self):
        return self.summary[['kcal', 'g protein', 'g fat']]
    
    @cached_property
    def enough(self):
        return all([(self.summary['kcal'] > target_kcal),
                   (self.summary['g_protein'] > target_protein),
                   (self.summary['g_fat'] > target_fat)
                   ]
                  )
    @cached_property
    def not_enough(self):
        if self.enough:
            return 0
        else:
            return 1
        
    @cached_property
    def nutrient_score(self):
        """ The nutrient score is 1 measure to check how far off the Recipe is from
        the minimum requirements (target_kcal, target_protein, target_fat). It is
        calculated as the sum of the relative deltas. So a value of 0 would mean
        that each target is met. Negative values (means it's below target) on
        seperate targets are multiplied by -1000 in order to force a high (bad) score.
        """
        scores = [0,0,0]
        scores[0] = (self.summary['kcal'] - target_kcal)/target_kcal
        scores[1] = (self.summary['g protein'] - target_protein)/target_protein
        scores[2] = (self.summary['g fat'] - target_fat)/target_fat
        for i, score in enumerate(scores):
            # heavily penalize negative scores:
            if score<0:
                scores[i] = -1000*score
        return sum(scores)

    # The optime Population requires an attribute called "dna" to exist on the
    # class to be used as individuals in the Genetic Algorithm. For Recipe, this
    # is just the kg_vec attribute:
    dna = kg_vec

As an example, create a random Recipe with maximum 10 ingredients:

In [ ]:
R1 = Recipe(max_ingredients=10)
R1.df[:12]

Hmmm, that looks delicious!
Just stir it all together and add some salt and pepper, but not too much or
you'll mess with the environmental impact and nutritional value:

In [ ]:
R1.summary

## Then use `optime` to optimize it
Now, we make a number of random `Recipe`s and use it as a starting population for the genetic algorithm. You can play with the `max_ingredients` parameter to limit your shopping time (set it to `None` - no maximum - if you feel adventurous!). Numbers lower than 15 generally don't yield very good results though. You gotta put in the work to save the planet!

We define the goals for the optimization, basically minimize all the environmental impact + nutrient_score (defined as the sum of 
the relative differences to their targets - so 0 is perfectly on target!)

In [ ]:
max_ingredients = 20

recipes = [Recipe(max_ingredients=max_ingredients) for _ in np.arange(20)]
pop = Population(recipes, goals_dict={
                                    'emissions':{'direction':'min','target':0},
                            'land_use':{'direction':'min','target':0},
                            'eutrophication':{'direction':'min','target':0},
                            'water_withdrawals':{'direction':'min','target':0},
                            'nutrient_score':{'direction':'min','target':0}
                                   })

Then, run the GA for a number of generations.
We can play with the number of generations (1st argument), the mutation probability (`mutprob`) - set this to a low number like 0.02 or something, because it is the probability of a mutation in the vector, and it has 200 values so you don't want too many ingredients to mutate incontrollably!
The `stop_on_steady_n` defines how many generations the mean and best values for each goal in the population must be constant before we decide it has converged.

Depending on the number of generations, this can run for a few minutes or more.

When the run is done, a plot is shown with the targets (0) and the mean and best values across the population for each goal (note that these don't occur within the same `Recipe` necessaririly) so we can check if there is convergence.
The 'best' line can be hidden under the 'target' line for the nutrient_score - which is a good thing!

In [ ]:
pop.run(100, mutprob=0.02, stop_on_steady_n=5, verbose=False)
pop.plot_progress()

Let's look at the Pareto Front after the optimization. According to the algorithm, these are the best Recipes at the moment, based on the 5 independent
optimization goals. Note that we didn't put a hard limit on nutritional value. Look at the nutrient_score: values < 1 are okay but higher values probably means at least 1 nutrient is below the target.

In [ ]:
best_recipes = pop.pareto().sort_values('nutrient_score', ascending=True)
best_recipes

We can arbitrarlity choose one of the Recipes above as the best, but let's just choose the one that does best for nutritional value - treat yo'self!

In [ ]:
chosen_recipe = best_recipes.iloc[0]['Individual']
chosen_recipe.df[0:max_ingredients-1]

In [ ]:
chosen_recipe.summary

Let's compare the top 5 `Recipe`s and see of we can find a common ingredient there.

In [ ]:
food_counter = []
top_recipes_n = 5
top_ingredients_n = 7
for i, recipe in enumerate(best_recipes.iloc[:top_recipes_n]['Individual']):
    print('\nRecipe number: ', i+1)
    print('Total values:')
    display(recipe.summary)
    print(f'Top {top_ingredients_n} ingredients (in kg):')
    food_counter = food_counter + list(recipe.df.index[:top_ingredients_n].values)
    print(recipe.df.index[:top_ingredients_n].values)

Now we can count how many times each ingredient occurs in this hall of fame of recipes:

In [ ]:
for food in set(food_counter):
    print(food, 'occurs ', food_counter.count(food), 'times')

So, that's it!
I don't know about your case, but for me, after onions, croissants appeared the most times! I was considering to go vegan, but now that I know that croissants are basically the second-best food to eat in order to save the planet, I will think again! I mean, I love onions, but not for breakfast...